In [58]:
import pandas as pd

In [59]:
songs = pd.read_csv("D:\\Projects\\airflow_docker\\data\\songs.csv")
streams = pd.read_csv("D:\\Projects\\airflow_docker\\data\\streams1.csv")
users = pd.read_csv("D:\\Projects\\airflow_docker\\data\\users.csv")

In [60]:
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.day
streams['listen_hour'] = pd.to_datetime(streams['listen_time']).dt.hour
full_data = streams.merge(songs, on = 'track_id').merge(users, on = 'user_id')

In [61]:
full_data.head()

,user_id,track_id,listen_time,listen_date,listen_hour,id,artists,album_name,track_name,popularity,...,instrumentalness,liveness,valence,tempo,time_signature,track_genre,user_name,user_age,user_country,created_at
0,26213,4dBa8T7oDV9WvGr7kVS4Ez,2024-06-25 17:43:13,25,17,91162,Panic! At The Disco,Pop n' Fresh,Don't Let the Light Go Out,0,...,0.00230,0.2710,0.1790,117.637,4,rock,Cathy Smith,39,United States,2024-06-05
1,6937,4osgfFTICMkcGbbigdsa53,2024-06-25 07:26:00,25,7,103402,Frank Ocean,Novacane,Novacane,81,...,0.00126,0.1600,0.3700,93.510,4,soul,Tommy Grant,61,United States,2024-10-16
2,21407,2LoQWx41KeqOrSFra089YS,2024-06-25 13:25:26,25,13,42552,Internal Rot,Grieving Birth,Harpooned,14,...,0.71800,0.5000,0.0689,122.842,3,grindcore,Garrett Ryan,34,United States,2024-08-05
3,47146,7cfG5lFeJWEgpSnubt4O4W,2024-06-25 18:17:50,25,18,50935,Rata Blanca,The forgotten Kingdom,The Forgotten Kingdom,20,...,0.00434,0.1840,0.5230,130.064,4,heavy-metal,Patrick Nash,44,United States,2024-01-21
4,38594,6tilCYbheGMHo3Hw4F22hF,2024-06-25 17:33:21,25,17,79664,Lafayette Leake,"Easy Blues (France, 1978) [Blues Reference]",Short Dressed Woman,49,...,0.07230,0.0853,0.7510,110.932,4,piano,Bonnie Walls DVM,69,United States,2024-11-07


In [62]:
# KPI 1: Hourly Unique Listeners
hourly_unique_listeners = full_data.groupby(['listen_date','listen_hour'])['user_id'].nunique().reset_index(name = 'unique_listener')

In [63]:
# KPI 2: Top Listened Artist of the Hour
artist_listen_counts = full_data.groupby(['listen_date','listen_hour','artists']).size().reset_index(name = 'listen_counts')
top_artist  = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist = top_artist.rename(columns = {'artists' : 'top_artists'})

In [64]:
# KPI 3: Listening Sessions per User per Hour
full_data['session_id'] = full_data['user_id'].astype(str) + '-' + full_data['listen_time'].astype(str)
sessions_per_user = full_data.groupby(['listen_date','listen_hour','user_id']).nunique('session_id').reset_index()
avg_sessions_per_user = sessions_per_user.groupby(['listen_date', 'listen_hour'])['session_id'].mean().reset_index(name='avg_sessions_per_user')

In [65]:
# KPI 4: Hourly Track Diversity Index
track_diversity = full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count']).reset_index()
track_diversity['diversity_index'] = track_diversity['nunique'] / track_diversity['count']

In [66]:
# KPI 5: Most Engaged User Group by Age per Hour
# Assuming age groups are segmented as 18-25, 26-35, etc.
users['age_group'] = pd.cut(users['user_age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])
user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')
most_engaged_group = user_group_engagement.loc[user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()].rename(columns={'age_group': 'most_engaged_age_group'})


C:\Users\Sasi\AppData\Local\Temp\ipykernel_26392\1894337850.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')


In [67]:
# Combine all KPIs into one DataFrame
final_kpis = hourly_unique_listeners.merge(top_artist, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(avg_sessions_per_user, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(track_diversity[['listen_date', 'listen_hour', 'diversity_index']], on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(most_engaged_group[['listen_date', 'listen_hour', 'most_engaged_age_group']], on=['listen_date', 'listen_hour'])

In [ ]:
final_kpis.head